<a href="https://colab.research.google.com/github/coldstalgia/SAAanalysis/blob/main/RTransferEntropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("RTransferEntropy")
library(RTransferEntropy)

# import dataset from final_table.csv
careggi_dataset <- read.csv(file = 'final_table.csv')

num_osp <- careggi_dataset[,'num_ospedalizzazioni']
press_mb <- careggi_dataset[,'pressione_mb']

mean_temp <- careggi_dataset[,'mean_temp']
max_temp <- careggi_dataset[,'max_temp']
max_wind <- careggi_dataset[,'max_wind']
humidity <- careggi_dataset[,'humidity']
var_press <- careggi_dataset[,'var_press']

# creating dataframe
table <- data.frame(num_osp, press_mb, mean_temp, max_temp, max_wind, humidity, var_press)


# display Transfer Entropy and p-value of given dataframe, with first column = num_ospedalizzazioni.
display_TE_pval <- function(dataframe, past_val=1, te_precision=4, p_precision=4){
# past_values = number of lagged values affecting the current value of x and y. Default is past_values = 1
# te_precision = precision of te value. Default is te_precision = 4
# p_precision = precision of p-value. Default is p_precision = 4

    # for each column except first
    for(col in colnames(dataframe[,2:ncol(dataframe)])){ 
        # for each past value considered effective
        for(k in 1:past_val){
            # lx, ly are number of lagged values affecting the current value of x and y
            result <- transfer_entropy(dataframe[col], dataframe['num_osp'], lx=k, ly=k, quiet=TRUE)

            # get te and p-value from an object of class transfer_entropy
            te <- round(coef(result)[1], te_precision)
            p_val <- round(coef(result)[7], p_precision)
            
            # formatting print function
            cmd <- sprintf("Variable: %s - (TE = %s, p-value = %s, k = %d)", col, te, p_val, past_val)
            print(eval(cmd))
        }
    }
}


# function that simply applies a given function D.Func to all possible pairs of columns from a given matrix X
FApply.Pairwise <- function(X, D.Func, lx, ly){
    n = seq_len(ncol(X))
    ff.TE.value = function(a, b) D.Func(X[,a], X[,b], lx=lx, ly=ly)

    return(outer(n, n, Vectorize(ff.TE.value)))
}

# print matrix with directional TE: X->Y & Y->X of given dataframe
print_TE_matrix <- function(dataframe, lx=1, ly=1, precision=4){
# lx = number of lagged values affecting the current value of x and y. Default is lx = 1
# ly = number of lagged values affecting the current value of x and y. Default is ly = 1
# precision = significant digit to show . Default is precision = 4

    TE.matrix<-FApply.Pairwise(dataframe, calc_te, lx, ly)
    rownames(TE.matrix)<-colnames(TE.matrix)<-colnames(dataframe)

    data <- round(TE.matrix, precision)
    print(data)
}
